# Common

In [1]:
#!pip install tqdm

In [2]:
import json
import time
from subprocess import Popen, PIPE, STDOUT
import math
import numpy as np
import pandas as pd
import hashlib
import requests
import os, sys, subprocess
from tqdm.contrib.concurrent import process_map
from tqdm.notebook import tqdm
tqdm.pandas()
import multiprocessing as mp
import shutil
import urllib.parse
import glob
import os
import sys
from ast import literal_eval
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
sys.path.append("./src/")

In [3]:
!mkdir -p Temp
!mkdir -p Result

In [4]:
current_path = os.getcwd()

In [5]:
def bracket_row(row):    
    s = row['data']
    index = min(s.find('.'), s.find('('))
    data = row['data']
    row['data'] = data[0:index]
    row['bracket'] = data[index:]
    return row

In [6]:
def adjust(text,n=7):
    text = str(text)    
    return " " * (n - len(text)) + text

In [7]:
def bracket_to_ct(tag, data, bracket, deltaG, negative_deltaG=True):    
    deltaG = deltaG.replace('(','').replace(')','')
    deltaG = float(deltaG)
    if(deltaG > 0 and negative_deltaG ): # negetive?!
        deltaG = -1 * deltaG
    stack = []
    index = np.zeros((len(bracket)), dtype = int)
    values = np.zeros((len(bracket)), dtype = int)
    for i in range(len(bracket)):
        index[i] = i + 1
        if(bracket[i] == '.'):
            values[i] = 0
        elif(bracket[i] == '('):
            stack.append(i)
        elif(bracket[i] == ')'):
            if(len(stack) == 0 ):
                print('structure error!')
            values[stack[-1]] = i + 1
            values[i]  = stack[-1] + 1
            stack.pop()
        else:
            print('structure error!')
    if(len(stack) != 0 ):
        print('structure error!')
    # body    
    ct = f"{adjust(len(data),6)} dG ={adjust(deltaG,10)} {tag}\n"   
    for i in range(len(bracket)):
        ct += f"{adjust(index[i],6)} {data[i]} {adjust(i,6)} {adjust((i+2)%(len(data)+1),6)} {adjust(values[i],6)} {adjust(index[i],7)}\n"
    return ct

In [8]:
def fasta_to_df(path):
    with open(path, 'r') as file:
        text = file.read()
    lines = [line for line in text.split('\n') if len(line) > 0]
    s = ''
    tags = []
    data = []
    for l in lines:
        if(l[0]=='>'):
            tags.append(l)        
            data.append(s)
            s = ''
        else:
            s += l    
    data.append(s)
    df = pd.DataFrame(
            {
                'tag': tags,
                'data': data[1:]
            })
    df['tag'] = df['tag'].apply(lambda x: x[1:])    
    return df

In [9]:
def df_to_fasta(df, path):
    lines = []
    df.apply(lambda row: lines.append(f">{row['tag']}\n{row['data']}\n"),axis=1)
    with open(path,'w') as file:
        file.write(''.join(lines))

In [10]:
def reformat(path):
    return path.replace('(','_').replace(')','_').replace('.','').replace(':','_')

In [11]:
def reformatCT(path):
    with open(path, 'r') as file:
        text = file.read()
    text = [l for l in text.split('\n') if len(l) > 0 ] # remove blank lines
    text = '\n'.join(text)
    text = text.replace("\t"," ")
    while("  " in text):
        text = text.replace("  ", " ")
    lines = [l for l in text.split('\n')]
    for i in range(len(lines)):
        if(lines[i][0] == " "):
            lines[i] = lines[i][1:]
        if(lines[i][-1] == " "):
            lines[i] = lines[i][:-1]
    text = '\n'.join(lines)
    return text

In [12]:
def get_ct_data(ct):
    ct = "\n".join(ct.split('\n')[1:])
    df = pd.read_csv(StringIO(ct), sep=" ", header=None)               
    nucleotide = df.iloc[:,1]
    index = df.iloc[:,5]
    values = df.iloc[:,4]
    return [nucleotide, index, values]

In [13]:
def ct2dot_bracket(path):
    [nucleotide, index, values] = get_ct_data(reformatCT(path))
    text = ''.join(nucleotide) + "\n"
    watch = []
    for i, v in zip(index,values):
        if(v == 0):
            text += '.'
        else:
            if( v not in watch):
                text += '('
                watch.append(i)
            if( v in watch):
                text += ')'
    return text

In [14]:
def is_nested(index, values):
    max_value = max(index) + 10 # inf
    for i, v in zip(index, values):
        if(v < max_value and v != 0):
            max_value  = v
        if(i >= max_value):
            max_value = max(index) + 10 # inf
        if(v > max_value):
            return False               
    return True

# Download dataset

In [15]:
input_genome_path = "GCA_022749755.1_ASM2274975v1_genomic.fna"

# Download data from Mirbase

In [16]:
directory = './Temp/miRBase_driven_data'
base = "https://www.mirbase.org/ftp/CURRENT"        
!rm -r {directory}
!mkdir -p {directory}

In [17]:
!wget {base}/aliases.txt.gz -P ./{directory}/       ; gzip -d ./{directory}/aliases.txt.gz 
!wget {base}/hairpin.fa.gz -P ./{directory}/           ; gzip -d ./{directory}/hairpin.fa.gz 
!wget {base}/hairpin_high_conf.fa.gz -P ./{directory}/ ; gzip -d ./{directory}/hairpin_high_conf.fa.gz 
!wget {base}/mature.fa.gz -P ./{directory}/            ; gzip -d ./{directory}/mature.fa.gz 
!wget {base}/mature_high_conf.fa.gz -P ./{directory}/  ; gzip -d ./{directory}/mature_high_conf.fa.gz
!wget {base}/miRNA.str.gz -P ./{directory}/            ; gzip -d ./{directory}/miRNA.str.gz 
!wget {base}/miRNA.xls.gz -P ./{directory}/            ; gzip -d ./{directory}/miRNA.xls.gz 
!wget {base}/organisms.txt.gz -P ./{directory}/        ; gzip -d ./{directory}/organisms.txt.gz

--2022-05-08 20:37:27--  https://www.mirbase.org/ftp/CURRENT/aliases.txt.gz
Resolving www.mirbase.org (www.mirbase.org)... 130.88.97.249
Connecting to www.mirbase.org (www.mirbase.org)|130.88.97.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480536 (469K) [application/x-gzip]
Saving to: ‘././Temp/miRBase_driven_data/aliases.txt.gz’

aliases.txt.gz      100%[===================>] 469.27K   455KB/s    in 1.0s    

2022-05-08 20:37:29 (455 KB/s) - ‘././Temp/miRBase_driven_data/aliases.txt.gz’ saved [480536/480536]

--2022-05-08 20:37:29--  https://www.mirbase.org/ftp/CURRENT/hairpin.fa.gz
Resolving www.mirbase.org (www.mirbase.org)... 130.88.97.249
Connecting to www.mirbase.org (www.mirbase.org)|130.88.97.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1547350 (1.5M) [application/x-gzip]
Saving to: ‘././Temp/miRBase_driven_data/hairpin.fa.gz’

hairpin.fa.gz       100%[===================>]   1.48M  1.02MB/s    in 1.4s    

2022-

In [18]:
mature = fasta_to_df(f'{directory}/mature.fa')
mature_high_conf = fasta_to_df(f'{directory}/mature_high_conf.fa')
mature['trim tag'] = mature['tag'].apply(lambda line: ' '.join(line.split(' ')[:2]))
mature['confidence'] = mature['trim tag'].isin(mature_high_conf['tag'])

In [22]:
mature['organism'] = mature['tag'].apply(lambda x: x[:3])
print(mature.shape)
mature.head(2)

(48885, 5)


,tag,data,trim tag,confidence,organism
0,cel-let-7-5p MIMAT0000001 Caenorhabditis elega...,UGAGGUAGUAGGUUGUAUAGUU,cel-let-7-5p MIMAT0000001,True,cel
1,cel-let-7-3p MIMAT0015091 Caenorhabditis elega...,CUAUGCAAUUUUCUACCUUACC,cel-let-7-3p MIMAT0015091,True,cel


In [23]:
organism = pd.read_csv(f'./{directory}/organisms.txt',sep='\t')
organism.columns = [c.replace('#','') for c in organism.columns] # remove sharp from columns
print(organism.shape)
organism.head(2)

(285, 5)


,organism,division,name,tree,NCBI-taxid
0,aqu,AQU,Amphimedon queenslandica,Metazoa;Porifera;,400682
1,nve,NVE,Nematostella vectensis,Metazoa;Cnidaria;,45351


In [24]:
items = list(organism['tree'].unique())
items.sort(key=len)
items

['Viruses;',
 'Mycetozoa;',
 'Alveolata;',
 'Metazoa;Porifera;',
 'Metazoa;Cnidaria;',
 'Viridiplantae;Chlorophyta;',
 'Viridiplantae;Embryophyta;',
 'Viridiplantae;Coniferophyta;',
 'Viridiplantae;Magnoliophyta;',
 'Metazoa;Bilateria;Deuterostoma;',
 'Chromalveolata;Heterokontophyta;',
 'Metazoa;Bilateria;Ecdysozoa;Nematoda;',
 'Metazoa;Bilateria;Lophotrochozoa;Annelida;',
 'Metazoa;Bilateria;Lophotrochozoa;Nemertea;',
 'Metazoa;Bilateria;Lophotrochozoa;Mollusca;',
 'Viridiplantae;Magnoliophyta;monocotyledons;',
 'Metazoa;Bilateria;Deuterostoma;Hemichordata;',
 'Metazoa;Bilateria;Deuterostoma;Echinodermata;',
 'Metazoa;Bilateria;Lophotrochozoa;Brachiopoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Hexapoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Crustacea;',
 'Metazoa;Bilateria;Lophotrochozoa;Platyhelminthes;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Chelicerata;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Mandibulata;',
 'Viridiplantae;Magnoliophyta;eudicotyledons;Poaceae;',
 'M

In [25]:
selectedTree = organism[organism['tree'].apply(lambda x: "Viridiplantae;" in x)]
print(selectedTree.shape)
selectedTree.head(5)

(86, 5)


,organism,division,name,tree,NCBI-taxid
66,cre,CRE,Chlamydomonas reinhardtii,Viridiplantae;Chlorophyta;,3055
67,pta,PTA,Pinus taeda,Viridiplantae;Coniferophyta;,3352
68,ppt,PPT,Physcomitrella patens,Viridiplantae;Embryophyta;,3218
69,smo,SMO,Selaginella moellendorffii,Viridiplantae;Embryophyta;,88036
70,ath,ATH,Arabidopsis thaliana,Viridiplantae;Magnoliophyta;eudicotyledons;Bra...,3702


In [26]:
selected = mature[mature['organism'].isin(selectedTree['organism'])]
print(selected.shape)
selected.head(1)

(10414, 5)


,tag,data,trim tag,confidence,organism
316,ath-miR156a-5p MIMAT0000166 Arabidopsis thalia...,UGACAGAAGAGAGUGAGCAC,ath-miR156a-5p MIMAT0000166,False,ath


In [37]:
df_to_fasta(selected,'./Temp/mature_microRNA_queries.fasta')

In [38]:
# use this cell for extracting str files for hairpin.fa
'''
tags = list(selected['tag'].apply(lambda x : x.split(' ')[0]))
with open(f'./{directory}/miRNA.str', 'r') as file:
    text = file.read()
text = text.split('\n')

result = ''
for i in range(0,len(text),8):
    if(text[i].split(' ')[0][1:] in tags):
        result += '\n'.join(text[i:i+8]) + "\n"        
with open(f'./high_conf_hairpin.str', 'w') as file:
    file.write(result)
''';

# Remove redundant

## cdhit-est

In [39]:
!./Software/cdhit/cd-hit-est -i ./Temp/mature_microRNA_queries.fasta  -o ./Temp/NR_mature_microRNA_queries.fasta \
    -c 1 -r 0 -G 1 -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS 0 \
    -AS 99999999 -s 0 -S 0

Program: CD-HIT, V4.8.1 (+OpenMP), Oct 23 2021, 21:45:39
Command: ./Software/cdhit/cd-hit-est -i
         ./Temp/mature_microRNA_queries.fasta -o
         ./Temp/NR_mature_microRNA_queries.fasta -c 1 -r 0 -G 1
         -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS 0 -AS 99999999
         -s 0 -S 0

Started: Sat Apr 16 19:11:23 2022
                            Output                              
----------------------------------------------------------------
total seq: 10414
longest and shortest : 28 and 17
Total letters: 222978
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 1M
Buffer          : 1 X 12M = 12M
Table           : 1 X 16M = 16M
Miscellaneous   : 0M
Total           : 30M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 96149440

comparing sequences from          0  to      10414
..........    10000  finished       5817  clusters

    10414  finished       6028  clusters



## reformat

In [40]:
with open('./Temp/NR_mature_microRNA_queries.fasta.clstr','r') as file:
    text = file.read()
lines = [line for line in text.split('\n') if len(line) > 0]
cluster = []
seqid = []
last_cluster = ""
for l in lines:
    if(l[0]=='>'):        
        last_cluster = l.replace('>Cluster ',"C")
    else:        
        cluster.append(last_cluster)
        seqid.append(l.split(', >')[1].split('...')[0])                
seq2cluster = pd.DataFrame({'seqid': seqid,'cluster': cluster})
print(seq2cluster.shape)
seq2cluster.head(2)    

(10414, 2)


,seqid,cluster
0,cst-miR11332,C0
1,stu-miR7994b-5p,C1


In [41]:
df = fasta_to_df("./Temp/mature_microRNA_queries.fasta")
df['accession'] = df['tag'].apply(lambda x : x.split(' ')[0])
seq2cluster = pd.merge(df,seq2cluster,how="inner",left_on='accession',right_on="seqid")
seq2cluster = pd.merge(seq2cluster, mature,how="inner",left_on='tag',right_on="tag")[['cluster','seqid','tag', 'confidence']]
print(seq2cluster.shape)
display(seq2cluster.head(2))
seq2cluster.to_csv('./Temp/seq2cluster.csv',index=False)

(10414, 4)


,cluster,seqid,tag,confidence
0,C5495,ath-miR156a-5p,ath-miR156a-5p MIMAT0000166 Arabidopsis thalia...,False
1,C1199,ath-miR156a-3p,ath-miR156a-3p MIMAT0031865 Arabidopsis thalia...,False


In [42]:
# todo: sorted first by cluster then by seqid
seq2cluster.sort_values("cluster").head(2)

,cluster,seqid,tag,confidence
9422,C0,cst-miR11332,cst-miR11332 MIMAT0044622 Cucumis sativus miR1...,False
7002,C1,stu-miR7994b-5p,stu-miR7994b-5p MIMAT0031188 Solanum tuberosum...,False


In [43]:
df = fasta_to_df("./Temp/NR_mature_microRNA_queries.fasta")
df['tag'] = df['tag'].apply(lambda x : x.split(' ')[0])
df = pd.merge(df,seq2cluster,how="inner",left_on='tag',right_on="seqid")[['cluster','data']]

lines = []
df.apply(lambda row: lines.append(f">{row['cluster']}\n{row['data']}\n"),axis=1)
print(df.shape)
with open('./Temp/BLASTn_queries.fasta','w') as file:
    file.write(''.join(lines))

(6028, 2)


# BlastN

!sudo apt-get install ncbi-blast+


In [45]:
!makeblastdb -in {input_genome_path} \
             -dbtype nucl \
             -out ./Temp/blastn_database



Building a new DB, current time: 04/16/2022 19:12:19
New DB name:   /home/jupyter/plant_microRNA_prediction/Temp/blastn_database
New DB title:  GCA_022749755.1_ASM2274975v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 70 sequences in 4.12774 seconds.


In [44]:
header = 'qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'

In [47]:
!blastn -query ./Temp/BLASTn_queries.fasta \
        -out ./Temp/BLASTn_result \
        -num_threads {mp.cpu_count()} \
        -db ./Temp/blastn_database \
        -word_size 7 \
        -penalty -3 \
        -reward 2 \
        -gapopen 5 \
        -gapextend 2 \
        -outfmt '6 qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'       

In [45]:
df_blastn = pd.read_csv('./Temp/BLASTn_result', sep='\t',header=None)
df_blastn.columns = header.replace("  "," ").split(" ")
print(df_blastn.shape)
df_blastn.head(2)

(495871, 27)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,gaps,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen
0,C5495,CM040446.1,1,20,1833813,1833794,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.005,37.4,...,0,100.0,1/-1,1,-1,minus,100,100,20,18270782
1,C5495,CM040446.1,1,20,5663467,5663486,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.005,37.4,...,0,100.0,1/1,1,1,plus,100,100,20,18270782


In [46]:
# alignment length adjustment
def blastn_adjust(row):
    if(row['sstrand'] == "plus"):        
        row['sstart'] = max(1, row['sstart'] - (row['qstart'] - 1))
        row['send'] = min(row['slen'], row['send'] + (row['qlen'] - row['qend']))
    if(row['sstrand'] == "minus"):        
        row['send'] = max(1, row['send'] - (row['qstart'] - 1 ))
        row['sstart'] = min(row['slen'], row['sstart'] + (row['qlen'] - row['qend'] ))
    return row
    
df_blastn = df_blastn.apply(lambda row: blastn_adjust(row), axis=1)

In [47]:
threshold = 3
df_blastn['Nonconformity'] = df_blastn['qlen'] - (abs(df_blastn['qend'] - df_blastn['qstart']) + 1) + df_blastn['gaps'] + df_blastn['mismatch']
df_blastn = df_blastn[df_blastn['Nonconformity'] <= threshold]
print(df_blastn.shape)
df_blastn.head(2)

(120752, 28)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen,Nonconformity
0,C5495,CM040446.1,1,20,1833813,1833794,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.005,37.4,...,100.0,1/-1,1,-1,minus,100,100,20,18270782,0
1,C5495,CM040446.1,1,20,5663467,5663486,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.005,37.4,...,100.0,1/1,1,1,plus,100,100,20,18270782,0


In [48]:
# remore redundancy and hold best one base of Nonconformity value
df_blastn = df_blastn.sort_values(["Nonconformity", "evalue"], ascending = (True, True))
df_blastn = df_blastn.drop_duplicates(subset=['sseqid','sstart', 'qseqid', 'send','sstrand'], keep='first')
df_blastn.to_csv('./Temp/filtered_out_blastn.csv')
print(df_blastn.shape)

(120751, 28)


In [49]:
#todo

# Result of the blastn to bed file

In [50]:
flanking_value = 200
df = df_blastn[['qseqid', 'sseqid', 'sstart', 'send', 'sstrand','slen']]
df['ones'] = 1

<ipython-input-50-cccdc8b60626>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ones'] = 1


In [51]:
def switch(row):
    if(row['sstart'] > row['send']):        
        temp = row['sstart']
        row['sstart'] = row['send']
        row['send'] = temp
    return row
df = df.apply(lambda row: switch(row), axis=1)

In [52]:
def convert(inp):
    if(inp == "plus"):
        return "forward"
    if(inp == "minus"):
        return "reverse"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['strand'] = df['sstrand'].apply(lambda x: convert(x))

In [53]:
def convert2sign(inp):
    if(inp == "plus"):
        return "+"
    if(inp == "minus"):
        return "-"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['sign'] = df['sstrand'].apply(lambda x: convert2sign(x))

In [54]:
df['hit_length'] = df.apply(lambda row: abs(row['send'] - row['sstart']) + 1 ,axis=1)

## convert sstart and send from location to index (range)

In [55]:
df['sstart'] = df['sstart'].apply(lambda x: x - 1)

In [56]:
df['downstream_flanking'] = df['sstart'].apply(lambda x:  flanking_value if x > flanking_value else x)

In [57]:
df['upstream_flanking'] = df.apply(lambda row:  flanking_value if (row['send']+flanking_value) <= row['slen'] else row['slen'] - row['send'],axis=1)

In [58]:
df['hit_start'] = df.apply(lambda row: row['downstream_flanking'] if row['sign'] == "+" else row['upstream_flanking'],axis=1)

In [59]:
df['hit_end'] = df.apply(lambda row: row['downstream_flanking'] + row['hit_length'] if row['sign'] == "+" else row['upstream_flanking'] + row['hit_length'],axis=1)

In [60]:
df['sstart'] = df['sstart'].apply(lambda x: max(x - flanking_value, 0))
df['send'] = df.apply(lambda row: min(row['send'] + flanking_value , row['slen']),axis=1)

In [61]:
df['tag'] = df.apply(lambda row: f">{row['sseqid']}:{row['sstart']}-{row['send']}({row['sign']})",axis=1)
df['reformated_tag'] = df['tag'].apply(lambda t: reformat(t))
df[['tag', 'reformated_tag', 'hit_start', 'hit_end']].to_csv('./Temp/hit_index_info.csv')#, index=False)

In [62]:
df['location_tag'] = df.apply(lambda row: f">{row['sseqid']}|{row['sign']}|{row['sstart'] + 1}-{row['send']}|{row['hit_start']+1}-{row['hit_end']}",axis=1)
df[['location_tag','qseqid']].to_csv('./Temp/pipe_seprated_location_list.csv',index=False,sep='\t')

In [63]:
df[['sseqid','sstart','send','strand','ones', 'sign']].to_csv('./Temp/extension_index.bed', 
        index=False, header=False, sep="\t")

# Extention


In [64]:
# !sudo apt-get install bedtools

In [70]:
!bedtools getfasta -fi {input_genome_path} -fo ./Temp/extended_original.txt -s -bed ./Temp/extension_index.bed
!rm input_genome.fna.fai

rm: cannot remove 'input_genome.fna.fai': No such file or directory


In [66]:
# todo: remove duplicated
'''
df = fasta_to_df("./Temp/extended.txt")
df = df.drop_duplicates(subset=['tag'], keep='first')
df_to_fasta(df,"./Temp/extended.txt")
len(df['tag'].unique())
''';

# Convert hit region to upper case and other region to lower case

In [71]:
ext = fasta_to_df('./Temp/extended_original.txt')
info = pd.read_csv('./Temp/hit_index_info.csv')
info['tag'] = info['tag'].apply(lambda x: x[1:])
print(info.shape)
info.head(2)

(120751, 5)


,Unnamed: 0,tag,reformated_tag,hit_start,hit_end
0,213130,CM040440.1:7209019-7209443(-),>CM0404401_7209019-7209443_-_,200,224
1,465602,CM040440.1:20398572-20398996(-),>CM0404401_20398572-20398996_-_,200,224


In [72]:
ext = ext.sort_values(by=['tag']).reset_index()
ext['help_tag'] = ext.apply(lambda r: r['tag'] + str(r.name),axis=1)
del ext['tag']

info = info.sort_values(by=['tag']).reset_index()
info['help_tag'] = info.apply(lambda row: row['tag']+ str(row.name),axis=1)
def redefined_tag(row):
    tag = row['tag']
    [sstart, send] = tag.split(':')[-1].split('(')[0].split('-')
    sstart = int(sstart) + 1
    sign = tag.split('(')[-1].split(')')[0]    
    return f"{tag.split(':')[0]}|{sign}|{sstart}-{send}|{row['hit_start']+1}-{row['hit_end']}"
info['tag'] = info.apply(lambda row: redefined_tag(row),axis=1)
ext = pd.merge(ext,info,how='inner', on='help_tag')

def emphasis_hit(row):
    seq = list(row['data'].lower())            
    s = row['hit_start']
    e = row['hit_end']
    seq[s:e] = list(''.join(seq[s:e]).upper())    
    return ''.join(seq)
    
ext['data'] = ext.apply(lambda row: emphasis_hit(row),axis=1)
df_to_fasta(ext[['tag','data']],"./Temp/extended_modified.txt")

# Protein coding elimination [Download nr]

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/nr.gz

# Protein coding elimination [Diamond]

In [ ]:
#!wget http://github.com/bbuchfink/diamond/releases/download/v2.0.13/diamond-linux64.tar.gz
#!tar xzf diamond-linux64.tar.gz

In [ ]:
!./diamond makedb --in ./NR/nr -d ./Temp/diamond_output

In [ ]:
!./diamond blastx -d ./Temp/diamond_output.dmnd\
                  -q ./Temp/extended_modified.txt
                  -o ./Temp/diamond_matches.tsv

In [73]:
dmn = pd.read_csv("./Temp/diamond_matches.tsv", sep='\t', header=None)
dmn.columns = 'qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore'.split(' ')
coding_seq = dmn['qseqid'].unique()

In [74]:
def clear(inp):
    if(inp[:9] == "reverse::"):
        return inp[9:]
    if(inp[:9] == "forward::"):
        return inp[9:]
    return inp
coding_seq = pd.Series(coding_seq).apply(lambda x : clear(x))

ext = fasta_to_df('./Temp/extended_modified.txt')
print(ext.shape)
ext = ext[~ext['tag'].isin(coding_seq)]
print(ext.shape)
df_to_fasta(ext,'./Temp/extended_modified_non_coding.txt')

(120751, 2)
(104385, 2)


# RNA 2d prediction

## Mfold

In [78]:
'''
# installation
!wget http://www.unafold.org/download/mfold-3.6.tar.gz
!tar -xvf ./mfold-3.6.tar.gz; rm ./mfold-3.6.tar.gz
%cd ./mfold-3.6
!./configure
!make
!make install
%cd ..
!sudo apt install texlive-font-utils
''';

In [79]:
#todo : add all hyperparameter(options) to GUI

In [80]:
counter = 0
base = "./Result/secondary_structure/mfold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df('./Temp/extended_modified_non_coding.txt')

for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/SEQ.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1    
    #if(counter >= 100):
      #  break

rm: cannot remove './Result/secondary_structure/mfold/': No such file or directory


In [81]:
#!mfold  SEQ="SEQ.FASTA" T=22 MAX=2

In [ ]:
%%capture
remove_lock = False
def run_mfold(tag):
    tag = reformat(tag)
    %cd {base + tag}
    !mfold  SEQ="SEQ.FASTA" T=22   
    if(not remove_lock):
        !find . -name "SEQ*" -not -name "*.ct" -not -name "*.pdf" -not -name "*SEQ.FASTA" -not -type d -delete
    %cd {current_path}

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count() - 5)  
    #pool.map(run_mfold, df['tag'].iloc[:100])
    pool.map(run_mfold, df['tag'])

In [ ]:
'''
base = "secondary_structure/mfold/"
for directory in glob.glob(f"{base}*"):    
    tag = directory[len(base):]
    ct_files = glob.glob(f'{directory}/*.ct')        
    try:
        ct_files.remove(f'{base}{tag}/SEQ.ct')
    except:
        print(directory)
        print(ct_files)
        print("*****************")
    for file in ct_files:        
        shutil.copy(file, './1.ct')
        #dot = ct2dot_bracket('./1.ct')
        #dot = dot.split('\n')
        #with open('./2.ct', 'w') as stream:
            #stream.write(bracket_to_ct(tag, dot[0] , dot[1] , "(0)"))        
        #ct1 = '\n'.join(reformatCT('./1.ct').split('\n')[1:])
        #ct2 = '\n'.join(reformatCT('./2.ct').split('\n')[1:])
        #if(ct1 != ct2):
            #print(file)
        ct = reformatCT('./1.ct')
        [nucleotide, index, values] = get_ct_data(ct)        
        #print(is_nested( index,  values))
        if(not is_nested( index,  values)):
            print("************")             
'''

In [84]:
print('done')

done


## Mxfold2

In [ ]:
#!wget https://github.com/keio-bioinformatics/mxfold2/releases/download/v0.1.1/mxfold2-0.1.1.tar.gz
#!pip3 install mxfold2-0.1.1.tar.gz
#!rm mxfold2-0.1.1.tar.gz

In [ ]:
!mxfold2 predict ./extended.txt > Result/secondary_structure/mxfold2_result.txt

In [ ]:
df = fasta_to_df('./Result/secondary_structure/mxfold2_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
df.head(2)

In [ ]:
base = "./Result/secondary_structure/mxfold2/"
!rm -r {base}
!mkdir -p {base}
for index, row in df.iterrows():    
    if(not os.path.exists(base + reformat(row['tag']))):
        os.makedirs(base + reformat(row['tag']))        
    tag = reformat(row['tag'])
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG)
        file.write(ct)    

## Vienna package

In [ ]:
#!wget https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_20_04/viennarna_2.4.18-1_amd64.deb -O viennarna.deb
#!sudo dpkg -i ./viennarna.deb
#!sudo apt-get -f install
#!rm viennarna.deb

In [ ]:
base = "./Result/secondary_structure/viennarna/"
!rm -r {base}
!rm ./Result/secondary_structure/viennarna_result.txt
!mkdir -p {base}

In [ ]:
%cd {base}
!RNAfold --jobs=0 --infile ../../Temp/extended_modified.txt  --noPS -T 22 > ../viennarna_result.txt
%cd {current_path}

In [ ]:
df = fasta_to_df('./Result/secondary_structure/viennarna_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
print(df.shape)
df.head(2)

In [ ]:
for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)      
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG, False)
        file.write(ct)    

In [ ]:
import glob
for file in glob.glob(f"{base}*.ps"):    
    f = file[len(base):-6] # _ss.ps 
    f = reformat(f)        
    shutil.move(file, f"{base}{f}/{f}.ps")    

## ContraFold

In [ ]:
#!wget http://contra.stanford.edu/contrafold/contrafold_v2_02.tar.gz
#!tar -xvzf contrafold_v2_02.tar.gz && rm contrafold_v2_02.tar.gz
#%cd contrafold/src
#!make clean
#!make 
# to file must changed to be complieable # utility.hpp and optimization.c++ files

In [ ]:
counter = 0
base = "./Result/secondary_structure/contrafold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df('./Temp/extended.txt')

for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/{tag}.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1    
    if(counter >= 10):
        break

In [ ]:
def run_contrafold(tag):
    tag = reformat(tag)    
    %cd Software/contrafold/src
    !./contrafold predict ../..{base[1:]}{tag}/{tag}.FASTA > ../..{base[1:]}{tag}/{tag}.dot
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'r') as file:
        text = file.read()
    text = [l for l in text.split("\n") if l[:len(">structure")] != ">structure"]    
    header = text[0]
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write('\n'.join(text[1:]))    
    !RNAeval  ../..{base[1:]}{tag}/{tag}.dot -T 20 > ../..{base[1:]}{tag}/{tag}.dotdg    
    with open(f"../..{base[1:]}{tag}/{tag}.dotdg", 'r') as file:
        text = file.read()
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write(header + "\n" + text)    
    
    df = fasta_to_df(f'../..{base[1:]}{tag}/{tag}.dot')
    df = df.apply(lambda row: bracket_row(row) , axis=1)        
    tag = reformat(df['tag'][0])
    with open(f'../..{base[1:]}{tag}/{tag}.ct','w') as file:
        bracket = df['bracket'][0].split(' ')[0]        
        deltaG = df['bracket'][0].split(' ')[1]
        ct = bracket_to_ct(df['tag'][0], df['data'][0], bracket, deltaG, False)
        file.write(ct)    
    #!rm ../..{base[1:]}{tag}/{tag}.dot
    #!rm ../..{base[1:]}{tag}/{tag}.dotdg
    !rm ../..{base[1:]}{tag}/{tag}.FASTA
    %cd {current_path}        

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count() - 1)  
    pool.map(run_contrafold, df['tag'].iloc[:10])

In [ ]:
'''path = 'secondary_structure/contrafold/AMWY020333941_469-893_-_/AMWY020333941_469-893_-_.dot'
!RNAeval  {path} -T 20 -v'''; 

# CTAnalizer

In [16]:
# only select those not ran before
base = "./Result/secondary_structure/mfold/"
df = fasta_to_df('./Temp/extended_modified.txt')
index_list =[]
for index, row in df.iterrows():    
    tag = reformat(row['tag'])    
    if(len(glob.glob(f'{base + tag}/*.ct')) != 0):
        index_list.append(index)
df = df.iloc[index_list,:]
print(df.shape)

(8916, 2)


In [17]:
from ct_analizer import get_row

In [18]:
def run(tag, path, extra):        
    #return get_row(tag, path, extra)
    try:
        return get_row(tag, path,extra)
    except Exception as e:
        print(str(e), tag)        
        return pd.Series()
        
def get_df_by_tag(tag , extra=0):           
    ct_files = glob.glob(f'{base}{reformat(tag)}/SEQ_*.ct')    
    return pd.Series(ct_files).apply(lambda path: run(tag, path,extra))    

In [19]:
d = get_df_by_tag("CM040442.1|-|21265562-21265985|201-224", extra=0)
#d.to_csv('Result/d.csv')#.iloc[0,:]['full seq visualization']

In [20]:
seq2cluster = pd.read_csv("Temp/seq2cluster.csv")
seq2cluster['tag'] = seq2cluster.groupby(['cluster'])['tag'].transform(lambda x: ','.join(x))
seq2cluster['seqid'] = seq2cluster.groupby(['cluster'])['seqid'].transform(lambda x: ','.join(x))
seq2cluster = seq2cluster.drop_duplicates()
tag2cluster = pd.read_csv('./Temp/pipe_seprated_location_list.csv',sep='\t')
tag2cluster['location_tag'] = tag2cluster['location_tag'].apply(lambda x : x[1:])
data = pd.merge(seq2cluster,tag2cluster,how='inner', left_on='cluster', right_on='qseqid')
data['Reference miRNA cluster'] = data['cluster']
data['Reference miRNA IDs'] = data['seqid']
data['Reference miRNA IDs and species'] = data['tag']
data = data[['location_tag','Reference miRNA cluster', 'Reference miRNA IDs', 'Reference miRNA IDs and species','confidence']]

# Apply on current data

In [21]:
rcols_boi = ['boi seq',
             'boi name',
             'boi dotbracket']

rcols_pre = ['precursor seq',
             'precursor name',
             'precursor dotbracket']

rcols_ref = ['Reference miRNA cluster',
             'Reference miRNA IDs',
             'Reference miRNA IDs and species']

rcols_hit = ['chromosome',
            'sign',
            'hit position on chromosome']
rcols = [*rcols_ref,
         *rcols_boi]

rcols_dg = [*rcols, 'delta G']

def selection(row):        
    global repeted
    tuple_row = tuple(row)            
    if(tuple_row not in repeted):
        repeted[tuple_row] = row.name
        return True
    return False

def boi_selection(row):    
    global repeted_boi        
    tuple_row_boi = tuple(row[rcols_boi])
    dg = row['delta G']
    if(tuple_row_boi not in repeted_boi):
        repeted_boi[tuple_row_boi] = { 
            "counter": 1,
            "dg": dg,
            "ref clusters": row['Reference miRNA cluster'],
            "ref ids": row['Reference miRNA IDs'],
            "ref species": row['Reference miRNA IDs and species'],
            "lock": False
        }
    else:            
        value =  repeted_boi[tuple_row_boi]
        value['counter'] += 1
        value['dg'] = min(value['dg'] , dg)
        value['ref clusters'] += "," + row['Reference miRNA cluster']
        value['ref ids'] += "," + row['Reference miRNA IDs']
        value['ref species'] += "," +  row['Reference miRNA IDs and species']                
        repeted_boi[tuple_row_boi] = value
        
            
def boi_JSC_cluster(row):    
    global repeted_boi_JSC    
    seq = row['boi seq']
    name = row['boi name']
    dotbracket = row['boi dotbracket']
    if(pd.isna(name) or name == "-"):
        return 
    name = name.split('|')    
    loction = name[0] + name[1] + name[2]
    hit = name[3]
    key = f'{seq}{dotbracket}{loction}'
    if(key in repeted_boi_JSC):        
        repeted_boi_JSC[key].append(hit)
    else:
        repeted_boi_JSC[key] = [hit]
        

def pre_JSC_cluster(row):    
    global repeted_pre_JSC    
    seq = row['precursor seq']
    name = row['precursor name']
    dotbracket = row['precursor dotbracket']
    if(pd.isna(name) or name == "-"):
        return 
    name = name.split('|')    
    loction = name[0] + name[1] + name[2]
    hit = name[3]
    key = f'{seq}{dotbracket}{loction}'
    if(key in repeted_pre_JSC):        
        repeted_pre_JSC[key].append(hit)
    else:
        repeted_pre_JSC[key] = [hit]
        
        
def hit_JSC_cluster(row):    
    global repeted_hit_JSC
    chrom = row['chromosome']
    sign = row['sign']
    hit = row['hit position on chromosome']                
    key = f'{chrom}{sign}'
    if(key in repeted_hit_JSC):        
        repeted_hit_JSC[key].append(hit)
    else:
        repeted_hit_JSC[key] = [hit]

In [22]:
chunksize = 1 * (10 ** 3)
max_workers = mp.cpu_count() - 4
num_terminal = 5 # acceptable_terminal_structures

repeted = {}
repeted_boi = {}
repeted_hit_JSC = {}
repeted_boi_JSC = {}
repeted_pre_JSC = {}
header = True
orders = None
arr = np.array_split(df['tag'], max(df['tag'].shape[0]//chunksize , 1))
for chunk in tqdm(arr):
    dfs = []
    for row in process_map(get_df_by_tag , chunk, tqdm_class=tqdm, max_workers=max_workers, chunksize=5):        
        dfs.append(row)
    chunk = pd.concat(dfs,axis=0)
    chunk = pd.merge(data, chunk, how='right', left_on = 'location_tag', right_on ='seq name')
    del chunk['location_tag']
    if(header):
        orders = chunk.columns
    for col in orders:
        if(col not in chunk.columns):
            chunk[col] = np.nan            
        
    for col in chunk.columns:
        if(col not in orders ):
            print(f"Error in {col}")        
    chunk = chunk.reindex(columns=orders)
    chunk = chunk.replace(np.nan, '-').replace('', '-')                    
    # delete repeated
    selected = chunk[rcols].apply(lambda row: selection(row), axis=1)
    chunk = chunk[selected]
    
    # cluster refs
    chunk[rcols_dg].apply(lambda row: boi_selection(row), axis=1)
    # boi JSC
    chunk[rcols_boi].apply(lambda row: boi_JSC_cluster(row), axis=1)
    # pre JSC
    chunk[rcols_pre].apply(lambda row: pre_JSC_cluster(row), axis=1)
    # hit JSC
    chunk[rcols_hit].apply(lambda row: hit_JSC_cluster(row), axis=1)
    
    chunk.to_csv("./Result/ct_analizer.csv", header=header, mode='a', index=False)    
    header = False

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

  0%|          | 0/1114 [00:00<?, ?it/s]

In [29]:
for v in repeted_boi_JSC.values():
    if(len(v) > 1):
        print(v)   

['63-83', '63-83']
['64-84', '64-84']
['96-116', '96-116']
['103-123', '103-123']
['62-82', '62-82']
['63-83', '63-83']
['93-112', '93-112']
['50-69', '50-69']
['30-49', '30-49']
['8-27', '8-27']
['13-32', '13-32']
['29-48', '29-48']
['18-37', '18-37']
['167-187', '167-187']
['15-35', '15-35']
['7-27', '7-27']
['192-212', '192-212']
['19-39', '19-39']
['201-221', '201-221']
['167-187', '167-187']
['2-22', '2-22']
['7-27', '7-27']
['167-187', '167-187']
['1-21', '1-21']
['195-215', '195-215']
['166-186', '166-186']
['16-36', '16-36']
['16-36', '16-36']
['16-36', '16-36']
['6-26', '6-26']
['16-36', '16-36']
['21-42', '21-42']
['24-45', '24-45']
['24-45', '24-45']
['21-42', '21-42']
['120-140', '120-140']
['187-207', '187-207']
['174-194', '174-194']
['174-194', '174-194']
['25-45', '25-45']
['175-195', '175-195']
['21-41', '21-41']
['194-214', '194-214']
['18-38', '18-38']
['3-23', '3-23']
['37-57', '37-57']
['50-70', '50-70']
['41-61', '41-61']
['5-25', '5-25', '5-25', '5-25']
['23-43',

In [ ]:
hit_JSC = 0.8
boi_JSC = 0.8
pre_JSC = 0.8

In [ ]:
for key in repeted_boi:
    value = repeted_boi[key]
    value['lock'] = False
    repeted_boi[key] = value    

In [ ]:
def isKeepCluster(row):
    global repeted_boi    
    dg = row['delta G']    
    if(row["boi name"] == "-"):
        return False
    tuple_row_boi = tuple(row[rcols_boi])    
    value = repeted_boi[tuple_row_boi]
    if(value['counter'] == 1):
        return True
    if(value['dg'] != dg):        
        return False
    if(value['lock']):
        return False
    value['lock'] = True
    repeted_boi[tuple_row_boi] = value
    return True


def makeCluster(row):            
    tuple_row_boi = tuple(row[rcols_boi])    
    value = repeted_boi[tuple_row_boi]
    if(value['counter'] != 1):
        for ref_c in ['ref clusters', 'ref ids', 'ref species']:
            value[ref_c] = value[ref_c].replace(' ,', ',').replace(', ', ',')                        
            value[ref_c] = value[ref_c].split(',')
            value[ref_c] = set(value[ref_c])
            value[ref_c] = ", ".join(value[ref_c])                    
        row['Reference miRNA cluster'] = value["ref clusters"]
        row['Reference miRNA IDs'] = value["ref ids"]
        row['Reference miRNA IDs and species'] = value["ref species"]
    return row


header = True
for chunk in tqdm(pd.read_csv("./Result/ct_analizer.csv", chunksize=10 ** 5)):
    chunk = chunk[chunk[rcols_dg].apply(lambda row:  isKeepCluster(row), axis=1)]
    chunk = chunk.apply(lambda row : makeCluster(row), axis=1)
    chunk.to_csv("./Result/ct_analizer_clustered.csv", mode='a', index=False)
    header = False

# Filters

# BlastX

In [ ]:
!makeblastdb -in ./NR/nr -dbtype prot -out ./NR/nr_database

#!head -n 100 ./Temp/extended_modified.txt > ./input_blastx.txt

!blastx -query ./input_blastx.txt \
        -db ./NR/nr_database \
        -out ./Temp/BlastX/blastx \
        -num_threads 20 \
        -evalue 1e-3 \
        -outfmt "6 qseqid sseqid qstart qend evalue bitscore score length frames qframe qcovs qcovhsp staxids"

blx = pd.read_csv('./Temp/BlastX/blastx', sep='\t', header=None)
blx.columns = 'qseqid sseqid qstart qend evalue bitscore score length frames qframe qcovs qcovhsp staxids'.split(' ')
coding_seq = blx['qseqid'].unique()

# psRNATarget

In [153]:
url = "https://www.zhaolab.org/psRNATarget/analysis"

payload='allowbulge=yes&curschema=s2&cutpos1=10&cutpos2=11&expect=5&function=3&gapextp=0.5&gapstartp=2&gup=0.5&hspsize=19&maxnummismatchinseed=2&misp=1&seedfactor=1.5&seedpos1=2&seedpos2=13&srna=&srna_content=%3Eath-miR156a%0D%0AUGACAGAAGAGAGUGAGCAC%0D%0A%3Eath-miR157a%0D%0AUUGACAGAAGAUAGAGAGCAC%0D%0A%3Eath-miR158a%0D%0AUCCCAAAUGUAGACAAAGCA%0D%0A%3Eath-miR398a%0D%0AUGUGUUCUCAGGUCACCCCUU%0D%0A%3Eath-miR398b%0D%0AUGUGUUCUCAGGUCACCCCUG%0D%0A%3Eath-miR398c%0D%0AUGUGUUCUCAGGUCACCCCUG%0D%0A%3Eath-miR834%0D%0AUGGUAGCAGUAGCGGUGGUAA%0D%0A%3Eath-miR390a%0D%0AAAGCUCAGGAGGGAUAGCGCC%0D%0A%3Eath-miR390b%0D%0AAAGCUCAGGAGGGAUAGCGCC&srna_uploaded=&target=&target_content=%3EAT1G27360.1%0D%0AAAGGTATCTATTTGCCTAGCCAGAGTTATATATAGGATTGATTGTCTAGTCTTTTCTTATATGATTTTTGTTCTCATTTACTAATCAAAGTTCTGCAAACTTGTAGTTGTTGTAGGATTTGTTGCTCTGGCTCTGGTGGTAGGTCTATGAAATCAACCCATATCGTGAATGGACTGCAACATGGTATCTTCGTCCCAGTGGGATTGGGAGCATTTGATCATGTCCAATCCGTCAAGGACTGAAGATGACAGCAAACAGCTACCTACTGAGTGGGAAATTGAAAAAGGTGAAGGAATTGAATCTATAGTTCCACATTTCTCAGGCCTTGAGAGAGTCAGTAGTGGCTCTGCCACCAGCTTCTGGCACACTGCTGTATCGAAAAGCTCACAGTCGACCTCTATCAACTCATCATCTCCCGAAGCCAAACGATGCAAGCTTGCATCAGAAAGTTCCCCTGGAGATTCTTGCAGCAACATAGACTTTGTCCAGGTGAAGGCTCCCACAGCTCTCGAGGTATCCGTTGCCTCAGCTGAATCAGATCTTTGTTTAAAACTAGGAAAGCGGACATACTCTGAAGAATACTGGGGTAGAAACAATAATGAAATTTCAGCGGTTTCTATGAAGTTGTTAACTCCATCTGTTGTCGCTGGGAAATCCAAATTGTGTGGTCAGAGCATGCCAGTCCCGCGTTGCCAAATTGATGGCTGTGAACTGGATCTCTCATCTGCTAAGGGTTATCATCGTAAGCACAAAGTCTGCGAAAAGCATTCAAAGTGCCCAAAAGTTAGCGTGAGTGGCCTGGAACGTCGGTTCTGCCAACAGTGTAGCAGGTTCCATGCTGTCTCTGAATTTGATGAGAAGAAACGAAGCTGCCGAAAACGTCTTTCTCATCATAATGCGAGGCGTCGTAAGCCACAAGGAGTATTTTCAATGAATCCCGAGAGGGTGTATGATCGAAGACAGCATACAAATATGTTGTGGAATGGGGTGTCCCTTAACGCGAGATCTGAAGAAATGTATGAATGGGGTAATAACACTTATGATACAAAGCCTAGACAAACGGAAAAAAGCTTTACTCTGAGCTTCCAGAGAGGTAATGGCTCTGAGGACCAGCTGGTTGCTAGTAGCAGCCGTATGTTCTCTACATCTCAAACCTCAGGTGGGTTCCCAGCAGGAAAGTCCAAGTTTCAACTTCATGGCGAAGATGTGGGAGAATACTCAGGAGTCCTCCATGAATCTCAAGATATCCACCGTGCTCTCTCTCTTCTGTCAACCTCTTCGGATCCCCTGGCCCAACCACATGTGCAGCCATTTTCTCTACTCTGTTCATATGATGTTGTACCAAAATAGATGAGTAAGTAATGTGTAATTTGTAAACCTGTTACTCAGTTGGTGGATACTTTTCCAAACCTATGATAAAAACCTCGTCCTAGATCCCGTTAAATGCCAAACTTTCGGCTACTATAACTATGTTATCGTTATCATTATCATTGTTTAACACCCT%0D%0A%3EAT1G27360.4%20%7C%20Symbols%3A%20%20%7C%20squamosa%20promoter-binding%20protein-like%2011%20(SPL11)%20%7C%20chr1%3A9501808-9503856%20FORWARD%0D%0ACTGGGTGAAACATAGAAAAGTTTCTCTTGCTCAAGTTAATGATAAAAGGGTGAGAGCAATAAACGCTGATAAGCCTTGTCTGGTCCTTGGAATTTTGAATTTTCTTTTTCTATCTTACTTATAGTATTGGTAGTTGAGGGTGTCGTCGATAAGTTGTTGTAGGATTTGTTGCTCTGGCTCTGGTGGTAGGTCTATGAAATCAACCCATATCGTGAATGGACTGCAACATGGTATCTTCGTCCCAGTGGGATTGGGAGCATTTGATCATGTCCAATCCGTCAAGGACTGAAGATGACAGCAAACAGCTACCTACTGAGTGGGAAATTGAAAAAGGTGAAGGAATTGAATCTATAGTTCCACATTTCTCAGGCCTTGAGAGAGTCAGTAGTGGCTCTGCCACCAGCTTCTGGCACACTGCTGTATCGAAAAGCTCACAGTCGACCTCTATCAACTCATCATCTCCCGAAGCCAAACGATGCAAGCTTGCATCAGAAAGTTCCCCTGGAGATTCTTGCAGCAACATAGACTTTGTCCAGGTGAAGGCTCCCACAGCTCTCGAGGTATCCGTTGCCTCAGCTGAATCAGATCTTTGTTTAAAACTAGGAAAGCGGACATACTCTGAAGAATACTGGGGTAGAAACAATAATGAAATTTCAGCGGTTTCTATGAAGTTGTTAACTCCATCTGTTGTCGCTGGGAAATCCAAATTGTGTGGTCAGAGCATGCCAGTCCCGCGTTGCCAAATTGATGGCTGTGAACTGGATCTCTCATCTGCTAAGGGTTATCATCGTAAGCACAAAGTCTGCGAAAAGCATTCAAAGTGCCCAAAAGTTAGCGTGAGTGGCCTGGAACGTCGGTTCTGCCAACAGTGTAGCAGGTTCCATGCTGTCTCTGAATTTGATGAGAAGAAACGAAGCTGCCGAAAACGTCTTTCTCATCATAATGCGAGGCGTCGTAAGCCACAAGGAGTATTTTCAATGAATCCCGAGAGGGTGTATGATCGAAGACAGCATACAAATATGTTGTGGAATGGGGTGTCCCTTAACGCGAGATCTGAAGAAATGTATGAATGGGGTAATAACACTTATGATACAAAGCCTAGACAAACGGAAAAAAGCTTTACTCTGAGCTTCCAGAGAGGTAATGGCTCTGAGGACCAGCTGGTTGCTAGTAGCAGCCGTATGTTCTCTACATCTCAAACCTCAGGTGGGTTCCCAGCAGGAAAGTCCAAGTTTCAACTTCATGGCGAAGATGTGGGAGAATACTCAGGAGTCCTCCATGAATCTCAAGATATCCACCGTGCTCTCTCTCTTCTGTCAACCTCTTCGGATCCCCTGGCCCAACCACATGTGCAGCCATTTTCTCTACTCTGTTCATATGATGTTGTACCAAAATAGATGAGTAAGTAATGTGTAATTTGTAAACCTGTTACTCAGTTGGTGGATACTTTTCCAAACCTATGATAAAAACCTCGTCCTAGATCCCGTTAAATGCCAAACTTTCGGCTACTATAACTATGTTATCGTTATCATTATCATTGTTTAACACCCT%0D%0A%3EAT1G32140.1%20%7C%20Symbols%3A%20%20%7C%20F-box%20family%20protein%20%7C%20chr1%3A11562722-11564813%20REVERSE%0D%0AATGACGATGATGTCCGACCTTTCACTTGATTTAGTCGAAGAGATATTGTGTAGGGTTCCGATAACTTCTCTTAAAGCAGTGAGATCTAGTTGCAAACTATGGAACGTTCTTTCCAAGAACCGGATTTTATGTAAAACAGAAGCTAGAAATCAGTTTTTAGGGTTCACGATAATGAATCATAGGCTTTATTCCATGAGATTCAATCTCCATGGAATCGGCCTCAATGAAAACAGTGAAGAGTTCATTGATCCATCTATAAAGCCAATAGGTAATTTACTTAATCAAGTCGAGATATCTAAAGTGTTTTATTGCGAAGGTTTATTGTTATGCGTCACAAGGAACCACTCAAGCAAGCTCGTAGTTTGGAACCCGTATTTGGGAGAAATTCGTTGGATCAAAACTAGGAATGATTACCACATAGGCGTTACATATGCTCTCGGGTACGACAACAACAAGAACCACATGATCTTGAGGTTTTTTTCTGAACAAGGCTACTACGAGATTTACGACATGAACTCTTCTGACTCATGGGATTGTTTTTATGGCATTCCCAACAAGGGGTTAAAATGTTATCAGCCCGGCGCGTCGTTAAATGGAAATGCTTATTTTTTGACTGAGGGAAGAGAAGTAATGGAAGGGTATGATTGCTTACTCGGTTTTGATTTTACAACAAAGAAATTTGGACCACTTCTTTCTTTGTCGTTTTCGCATGATTTTATAGAGACTGGGAGACTATCTTGTGTTAAAGGAGAGAAACTTGCGGTCTTATATCAGCGCTGCTATACCTATGAGATGGAGGTATGGGTGACAACTAAGATAGAGCCGAATGCGGTGTCATGGAGCAAATTCTTAGCAGTTGAAATGGAACCACTCACTAGTCTAAAGTTTAACGATGATTCTGGCAGTTTCTTCATTGACGAAGAGAAGAAAATCGTCGTGGTTTTTGATATAGACGAATCTGAACGCAACAATACGGCTTACATCATTGGAGATTATGGATGCTTGAAAGAAGTTGATCTTGATGAAGTTGTGAACCCACAAGAATCTGTGGAGGTCGGAGACCGCATTTATTCTTTTTCACCATTTGTGTGCTCTTGCTCTTATGTTCCAAGTTTAGTGAAATTTAAAGAAGATGCAGAACATGAAAGGAAAGATAAGAAGAGGAAGAGTAAGAGGAAGCGAACCAACAAGGATGGATATGATTTTATACTGTGTTTTGATTTTACAACCGAGAGATTTGGACAGATTCTTCCTCTGCCGTTTAAACATTCTTTTAGGGATACTTGGACTCTATCTTCTGTTAAAGAAGAGAAACTTGCAGTCGCAGTGTTATACTGGAAAAATACATGTGTGATGATAGAGATATGGATGACAATTAAGATTGATCCTAATGTCGAGTCGTGGAGCAAATTCTTAAGAGTTGATAGGAAACCATGCATTGATCTTCGCTTTGATGATCGTAATGACAGTTTTTTCATTGACGAAGAGAAGAAAGTTGTCGTGTTTTTTAGTTCAGACAAAGTTAAAACCTCTACGGCTTACGTCATTGGAGATAATAGATATTTGAGAACAGTGGATCTTGAAAAAGCTGCAAACTCCCAAGAATCTGTGGAGGTCGGAGAACGCGTGTATTGTTTTTCGCCGCTTGTGTGCTCTTGCTCTTATTATGTTCCAAGTTTGGTGAAAATCAACCACAATGCAGGACGCAAAAGGAAAGAGAAGAAGACGAAGCGCAAAAGTAAAGACAAGCAGATGAAACTAAGCAACAAGGTGTAA%0D%0A%3EAT2G42200.1%20%7C%20Symbols%3A%20%20%7C%20squamosa%20promoter-binding%20protein-like%209%20(SPL9)%20%7C%20chr2%3A17594485-17596708%20FORWARD%0D%0AACCACTCTCGTCTCTTTCTTTTTTCCTTCTGTTCTGTTTCTCTCTCTAAACCCAAAACAGTCAAAATCAGGGAAGCCGAAATTTTCTTTGCTTTCTTCTCCTTTGGTCCTTTCTTTAAACCCGAGACAGTTAGGTTTGTGTGAGAGAGAGAATGATGAGTAAAACCCTTTCTGTCTGAGTAAGAGGAAACCAACATGGAGATGGGTTCCAACTCGGGTCCGGGTCATGGTCCGGGTCAGGCAGAGTCGGGTGGTTCCTCCACTGAGTCATCCTCTTTCAGTGGAGGGCTCATGTTTGGCCAGAAGATCTACTTCGAGGACGGTGGTGGTGGATCCGGGTCTTCTTCCTCAGGTGGTCGTTCAAACAGACGTGTCCGTGGAGGCGGGTCGGGTCAGTCGGGTCAGATACCAAGGTGCCAAGTGGAAGGTTGTGGGATGGATCTAACCAATGCAAAAGGTTATTACTCGAGACACCGAGTTTGTGGAGTGCACTCTAAAACACCTAAAGTCACTGTGGCTGGTATCGAACAGAGGTTTTGTCAACAGTGCAGCAGGTTTCATCAGCTTCCGGAATTTGACCTAGAGAAAAGGAGTTGCCGCAGGAGACTCGCTGGTCATAATGAGCGACGAAGGAAGCCACAGCCTGCGTCTCTCTCTGTGTTAGCTTCTCGTTACGGGAGGATCGCACCTTCGCTTTACGAAAATGGTGATGCTGGAATGAATGGAAGCTTTCTTGGGAACCAAGAGATAGGATGGCCAAGTTCAAGAACATTGGATACAAGAGTGATGAGGCGGCCAGTGTCGTCACCGTCATGGCAGATCAATCCAATGAATGTATTTAGTCAAGGTTCAGTTGGTGGAGGAGGGACAAGCTTCTCATCTCCAGAGATTATGGACACTAAACTAGAGAGCTACAAGGGAATTGGCGACTCAAACTGTGCTCTCTCTCTTCTGTCAAATCCACATCAACCACATGACAACAACAACAACAACAACAACAACAACAACAACAACAACAATACATGGCGAGCTTCTTCAGGTTTTGGCCCGATGACGGTTACAATGGCTCAACCACCACCTGCACCTAGCCAGCATCAGTATCTGAACCCGCCTTGGGTATTCAAGGACAATGATAATGATATGTCTCCTGTTTTGAATTTAGGTCGATACACCGAGCCAGATAATTGTCAGATAAGTAGTGGCACGGCAATGGGTGAGTTCGAGTTATCTGATCACCATCATCAAAGTAGGAGACAGTACATGGAAGATGAGAACACAAGGGCTTATGACTCTTCTTCTCACCATACCAACTGGTCTCTCTGACTTGTCTTTGCATCAGAGAATCTTCTTACAATGAACGATTCTGCAATATCTTATCTTTTTGCTTCTTTGTTTATTCTGTTATCTGCTATCAATAAACCAGACAATTGTTGCCAGATAATGGCTTTTGATTTTGATTTGTTGTTTTATCTCCATGAAAATCCAAGTTATGAGATCAGATT%0D%0A%3EAT1G49910.1%20%7C%20Symbols%3A%20%20%7C%20WD-40%20repeat%20family%20protein%20%2F%20mitotic%20checkpoint%20protein%2C%20putative%20%7C%20chr1%3A18482693-18485143%20FORWARD%0D%0AATGACTTTGGTGCCGGCCATTGGTCGCGAGCTCTCGAATCCACCGTCCGATGGGATTTCTAATCTTCGATTTTCTAATAACAGCGATCATTTACTAGTCTCTTCATGGGATAAGAGTGTAAGATTGTATGATGCGAACGGCGATTTGATGAGAGGGGAGTTTAAACATGGTGGAGCGGTACTCGATTGCTGCTTCCATGATGATTCTTCTGGATTCAGTGTTTGCGCCGATACTAAAGTTAGAAGAATTGACTTCAATGCTGGCAAAGAAGACGTTTTGGGTACGCATGAGAAGCCAGTTCGATGTGTTGAGTATTCTTATGCTGCAGGGCAAGTGATCACTGGAAGTTGGGATAAAACGATTAAATGTTGGGATCCAAGAGGTGCAAGTGGGACTGAGCGCACACAGATTGGAACTTATATGCAACCTGAGCGTGTTAACTCTCTTTCTCTTGTTGGAAATCGTTTGGTAGTGGCAACAGCAGGAAGGCATGTCAACATTTATGATCTTAGAAATATGTCCCAGCCTGAGCAAAGAAGAGAGTCCTCACTTAAATACCAGACAAGATGTGTACGTTGTTATCCCAACGGAACAGGATATGCCCTTAGCTCTGTTGAAGGGAGGGTTTCAATGGAGTTTTTTGATCTATCAGAAGCTGCTCAGGCTAAAAAATATGCTTTCAAATGTCACCGGAAATCAGAGGATGGAAGGGACATTGTCTACCCTGTAAATGCAATTGCTTTCCATCCGATTTATGGCACTTTTGCTTCCGGAGGCTGTGATGGTTTTGTCAACATTTGGGACGGTAACAATAAGAAGAGGCTTTATCAGTACTCTAAGTATCCAACAAGTATTGCGGCGCTGTCATTCAGCCGAGATGGTGGATTACTGGCTGTTGCTTCTAGTTACACGTTTGAAGAGGGAGACAAACCGCATGAACCGGACGCCATCTTTGTTAGAAGTGTTAATGAAATTGAAGTGAAACCGAAACCCAAAGTATACCCAAATCCCCCGGTATAGTCAAGAAATAATGGAATGAGCAGAGTCAAATTCGACTTGTGTGTTGTTGTATTGTAGCACTTGAAAGTGAGTTATAAAATCTTATTTTGGCTGTAAAGTGAAATGTGAACGTTATAATGGCTTTCGAATCTGAGATGGTGTTCCATTTACTCTCTGGGTTGCCTCCAATTTTTCTTTTAGGACCAACTATCTTATTTTTACCTT%0D%0A%3EAT1G48460.1%20%7C%20Symbols%3A%20%20%7C%20similar%20to%20unknown%20protein%20%5BArabidopsis%20thaliana%5D%20(TAIR%3AAT5G63040.2)%3B%20similar%20to%20Os01g0704200%20%5BOryza%20sativa%20(japonica%20cultivar-group)%5D%20(GB%3ANP_001044004.1)%3B%20similar%20to%20hypothetical%20protein%20MtrDRAFT_AC124952g33v1%20%5BMedicago%20truncatula%5D%20(GB%3AABE93586.1)%3B%20contains%20domain%20Multidrug%20resistance%20ABC%20transporter%20MsbA%2C%20N-terminal%20domain%20(SSF90123)%20%7C%20chr1%3A17915014-17916968%20FORWARD%0D%0AAGAAAACAATTCCAAAAAATAAAATGTCAGAAAAGAATTTTCTTTTAGAATAAAGACAGTGAAGAGATTTATTTCAAAGCCTGGGTTTAAGCTGCTGAGAGAACACAAAAAACCCTAACAAAAATGGAATCGAAAGCAATTTGCTTAGGGTTTCTTCCTCCAAGACTTCGATTTTCATCTCCACGTTTACTCTCTCTTCCTCCTTCTCCTCCTGCTTCTTCCACATTTGCGACGCGTCACAAACTTGATTCCAGACAAACCCTCCTTTGGAACAAACCGCAATTGAGCCGAGTTCGTGTAGCGTGTTCTTCTTCTCAATCTGACTCAAGACCTGAGAAGAAGCAATCGGATAAGAGTAACTATGCTCGAGCTGAGCTGTTCCGTGGGAAATCAGGTTCTGTTTCTTTCAATGGTCTGACTCATCAGCTGGTTGAAGAAAGTAAACTGGTTTCAGCTCCGTTTCAAGAAGAGAAAGGTTCTTTCTTGTGGGTTTTGGCTCCTGTTGTTTTGATTTCTTCGTTGATTCTTCCTCAGTTCTTTCTAAGTGGTATCATTGAAGCTACCTTCAAAAACGACACTGTTGCTGAAATTGTTACTTCTTTTTGCTTTGAGACGGTGTTTTATGCTGGTCTTGCGATATTCCTGTCTGTGACTGACCGAGTGCAGAGGCCGTACTTAGACTTCAGCTCCAAGAGATGGGGTCTGATCACTGGACTGAGGGGATACCTTACGTCTGCATTCCTCACGATGGGTTTAAAAGTTGTAGTTCCCGTATTTGCTGTTTACATGACTTGGCCAGCTCTTGGAATAGATGCTTTGATTGCAGTGCTTCCTTTCTTGGTTGGCTGTGCAGTTCAAAGAGTTTTCGAGGCTCGGCTTGAAAGACGTGGCTCATCCTGTTGGCCCATTGTTCCAATAGTCTTTGAGGTGTATAGGCTGTATCAGGTGACAAGAGCAGCGACTTTTGTTCAGAGGCTGATGTTTATGATGAAAGATGCGGCAACGACTGCTGAAATAACAGAGCGAGGAGTTGCACTAGTTGGTTTGGTTGTGACTTTGCAGTTTCTAGCTGTTATGTGTCTCTGGTCGTTTATCACTTTTCTTATGCGCCTCTTTCCTTCTAGACCTGTAGGTGAAAACTACTAGATCTCAGTGTTTAGTGATTGTTAGATGTAGCCAAATCCCATCGGTTTTGTTTTGTTTCTGTGTTCATTTCAGTAGTAATGAATTGTATTAAGTCACTTTAAGAATTGGTTGATCATGTGAAATGAGAATTGGCTGGAAATGTTATAGAACG%0D%0A%3EAT5G50570.2%20%7C%20Symbols%3A%20%20%7C%20squamosa%20promoter-binding%20protein%2C%20putative%20%7C%20chr5%3A20599309-20601106%20REVERSE%0D%0AAAAAAGGACAAATCTTGATATTGCTTTGATTGCTGTTGTGTATGTATGTGTTTTTATAGTGAGAGAAGAAAAAAAAGCACAATCTTTGAATGGACTGGAATTTCAAACTTAGCTCTGGTTATTTATCTGGATTCGATCAAGAACCAGATTTATCACCAATGGATGGTTCGATCTCGTTTGGTGGGTCGTCACAGTCAAAAGCGGATTTTTCATTTGATCTAAAACTTGGAAGAAACATTGGAAACTCTTCCTCTGTTTTTGGTGATACAGAGCAAGTGATTAGTCTTAGTAAGTGGAAAGATAGTGCTTTAGCTAAACCAGAAGGTTCAAGAAGCTCGAGTTCAAAGAGAACAAGAGGGAATGGTGTTGGAACCAACCAGATGCCGATTTGTCTTGTTGATGGATGTGATTCTGATTTTAGTAATTGTAGAGAGTATCATAAGAGACATAAAGTTTGTGATGTTCATTCAAAAACTCCTGTGGTTACTATTAATGGTCATAAACAGAGGTTTTGTCAACAATGCAGCAGGTTTCATGCTTTGGAGGAGTTTGATGAAGGGAAGAGAAGTTGTAGGAAACGTCTTGATGGACATAATCGAAGACGACGGAAGCCGCAGCCTGAACATATCGGTCGTCCTGCCAACTTCTTTACGGGTTTTCAAGGTAGCAAATTGCTAGAGTTTTCTGGTGGTTCACATGTGTTTCCAACTACATCTGTGTTGAACCCGAGCTGGGGAAATAGTCTTGTAAGCGTTGCTGTAGCCGCCAATGGTTCGAGTTATGGGCAGAGCCAGAGCTATGTTGTTGGTTCTTCTCCTGCAAAGACAGGGATAATGTTTCCAATCTCTTCTTCTCCAAACAGTACCAGAAGCATAGCAAAACAATTCCCTTTCTTGCAAGAAGAAGAAAGCTCGAGAACCGCATCGTTGTGTGAGAGAATGACGAGTTGCATCCATGACTCTGATTGTGCTCTCTCTCTTCTGTCATCCTCCTCGTCGTCAGTCCCTCATTTGCTTCAACCACCACTTTCTTTGTCCCAAGAAGCAGTTGAGACAGTTTTTTACGGGTCGGGATTGTTTGAGAATGCGAGTGCAGTCTCTGATGGATCGGTTATATCCGGTAACGAGGCTGTCCGTCTTCCGCAGACATTCCCGTTTCATTGGGAGTAGTAGAAGAAGAAGTAGGTAGATAGATAGAATCAGAAAGATCTATTTGTGTCTCTTCTCTTCTCCCTCATTTTTCAATGTTCTTTATCATCATCATTGTTCTTGTTAACACTACAAGAAATATGGACATTCTTAACACACCGAAAACGCTATAATAACGTTTACATAGCGGATTCATAAACGCTGTGTTTGCCGGAGCTATCTTAGAGTGGTCACATACAATAGCGTTTCTTGCTATGCTATTAAATGTTCACATATTATGGCGTAAAAGAAATGCTTTGATTTCCTTTGTTATTGCACAATTTTGATGTTATACTTTTGTAACTCTTTTTAAGGGCTATAAACTATTATTTTGTAGCTATATTTTATAGGCTATGATCTGATATGTTGTAGCTTTATTTTTTTGGCTATAAATCTATAAACAGCCTATAAGTTTGGGACATTTTGTTACACATTTGCAAGAAACCTCTTTTTG%0D%0A&target_uploaded=&top=200'
headers = {
  'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'Upgrade-Insecure-Requests': '1',
  'Content-Type': 'application/x-www-form-urlencoded',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'session=eyJzaW1wbGVwYWdlIjpmYWxzZX0.YhEQeA.N4sPemlWsAA1q7EwbfdkVXJbqs8'
}

response = requests.request("POST", url, headers=headers, data=payload)
type(response)

requests.models.Response